In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from fuzzywuzzy import fuzz
import Levenshtein

In [2]:
df = pd.read_csv('Hemolysis/Cleaned_hemolytic_data.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.columns = [c.lower() for c in df.columns]
df.sequence = df.sequence.str.upper()

In [16]:
def knn_toxicities(peptide_row, df, k, min_uncertainty):
    sequence = peptide_row.sequence
    distances = df[df.uncertainty < min_uncertainty][df.sequence != sequence].sequence.apply(lambda x: Levenshtein.distance(x, sequence))
    bestk = df.loc[distances.sort_values().iloc[:k]]
    avg_toxicity = np.mean(bestk.log10_hc50)
    return avg_toxicity


In [17]:
for i, row in df.iterrows():
    bestk = knn_toxicities(row, df, 5, 0.1)
    df.at[i, 'bestk'] = bestk

/Users/zswitten/.pyenv/versions/2.7.12/lib/python2.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
df[['log10_hc50', 'bestk']].corr()

,log10_hc50,bestk
log10_hc50,1.00000,0.03919
bestk,0.03919,1.00000


In [7]:
df.uncertainty.describe()

count    1377.000000
mean        0.322304
std         0.457971
min         0.000000
25%         0.000000
50%         0.000000
75%         0.616395
max         2.888139
Name: uncertainty, dtype: float64